The old locker subgraph does not account for relocks and will not let you query the balance of a user at a certain block or timestamp (an improvement left for the subgraph that will track the new locker contract). However it can still be used to retrieve a list of addresses who have interacted with the contract. That address can then be used to query the locker contract at a certain block with multicall to get their balances.

The present notebook does that.

In [1]:
import requests
import pandas as pd
import json
import brownie
import requests
from brownie import Contract

In [2]:
GRT_QUERY_ENDPOINT = "https://api.thegraph.com/subgraphs/name/convex-community/locker"
def grt_query(query):
    r = requests.post(GRT_QUERY_ENDPOINT, json={'query': query})
    return r.json()

In [3]:
users = []
count = 0
last_users = []
last_amount = 1867562036033395438700658 # max_value (can't use 0 and asc because too many people have migrated from the new contract at this stage)
while True:
    # we have over 6000 address so we can't use the usual first / skip params to paginate and instead need to filter a bit further 
    # as per : https://thegraph.com/docs/en/developer/graphql-api/#example-4
    query = f"{{ users(first: 1000 where: {{totalLocked_lte: \"{last_amount}\"}} orderBy: totalLocked orderDirection: desc) {{ id totalLocked }} }}"
    res = grt_query(query)
    count += 1
    page_users = [entry['id'] for entry in res['data']['users']]
    if page_users == last_users:
        break
    last_users = page_users
    last_amount = [entry['totalLocked'] for entry in res['data']['users']][-1]
    print(f"{count}: {last_amount}")
    users += page_users

1: 1428104411709065755908
2: 406094408296199524468
3: 122753910974405529968
4: 25344307580593902703
5: 0
6: 0


In [12]:
unique_users = list(set(users))

In [ ]:
#brownie.network.disconnect()
brownie.network.connect("mainnet-fork")

In [6]:
locker_contract = Contract.from_explorer("0xD18140b4B819b895A3dba5442F959fA44994AF50")

Fetching source of 0xD18140b4B819b895A3dba5442F959fA44994AF50 from api.etherscan.io...


/home/ben/.local/lib/python3.10/site-packages/brownie/network/contract.py:1236: BrownieCompilerWarning: 0xD18140b4B819b895A3dba5442F959fA44994AF50: Locally compiled and on-chain bytecode do not match!
  warnings.warn(


In [7]:
multi = brownie.multicall(address="0x5BA1e12693Dc8F9c48aAD8770482f4739bEeD696", block_identifier=14318930)

In [8]:
multi

In [9]:
with multi:
    print(brownie.multicall.address)
    balances = [locker_contract.lockedBalanceOf(user) for user in users] # update block here to snapshot block

0x5BA1e12693Dc8F9c48aAD8770482f4739bEeD696


In [11]:
final_list = {user: balances[i] for i, user in enumerate(users)}
final_list

{'0x3ff634ce65cdb8cc0d569d6d1697c41aa666cea9': 1160897623637994419234011,
 '0x9c5083dd4838e120dbeac44c052179692aa5dac5': 1000000002647663906037535,
 '0xa22eb3338dfd69458513a1f6d4742ab29f7ef333': 1000000000000000000000000,
 '0xf96da4775776ea43c42795b116c7a6eccd6e71b5': 835409131233770474563531,
 '0xdfc95aaf0a107daae2b350458ded4b7906e7f728': 814369958039330400003503,
 '0xbec5e1ad5422e52821735b59b39dc03810aae682': 670334841143543479717332,
 '0x1e1300eeaf333c572e4fc0133614291fa9d0df8b': 513274799293735990472388,
 '0xe001452bec9e7ac34ca4ecac56e7e95ed9c9aa3b': 491524033583539156969181,
 '0xa0f75491720835b36edc92d06ddc468d201e9b73': 400000000000000000000000,
 '0x012144ba349d6ae5b68581b29ea4d42b1f43fdb7': 370925119846412439337670,
 '0x9a67f1940164d0318612b497e8e6038f902a00a4': 366116714664631067148745,
 '0x3f3e305c4ad49271ebda489dd43d2c8f027d2d41': 299998002439002564629955,
 '0xf3d9b8f6af674e82943b4685728b00d13a62fc9e': 283204736261756202274296,
 '0xaba85673458b876c911ccff5e3711bcedb3b4f56': 2